In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
import urllib
import numpy
import pandas as pd
from pyproj import Proj,transform
import numpy as np
import json

#한국환경공단_에어코리아_대기오염통계 현황

In [ ]:
key = 'd7pOtZHjihMLv0obKwPcxzlRNCwfcmnNMjNVwNc08mxUQMg%2B4EYxcVmqBEjZbjgpB2PpHj5ExgzjIjkoz5IHBQ%3D%3D'
contentType = 'xml'
sido = urllib.parse.quote('서울') #한글을 쿼리문자열로 변환 후 검색
url = 'http://apis.data.go.kr/B552584/ArpltnStatsSvc/getCtprvnMesureSidoLIst?sidoName='+sido+'&searchCondition=DAILY&pageNo=1&numOfRows=100&&returnType=xml&serviceKey='+key
res = req.urlopen(url)
soup = BeautifulSoup(res.read().decode('utf-8'), features="xml")
print(url)
print(soup)


In [ ]:
busNum = '360'
key = '%2BjzsSyNtwmcqxUsGnflvs3rW2oceFvhHR8AFkM3ao%2Fw50hwHXgGyPVutXw04uAXvrkoWgkoScvvhlH7jgD4%2FRQ%3D%3D'
url1 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey='+key+'&strSrch='+busNum
res = req.urlopen(url1)

soup = BeautifulSoup(res.read().decode('utf-8'), features="xml") #xml은 무조건 utf-8로 인코딩해야 함.

busRouteId = None
for itemList in soup.find_all('itemList') :
    busRouteId = itemList.find('busRouteId').string
    busRouteNm = itemList.find('busRouteNm').string
    if busRouteNm == busNum :
        break

url2 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey='+key+'&busRouteId='+busRouteId
res = req.urlopen(url2)
soup = BeautifulSoup(res.read().decode('utf-8'), features="xml")

busPos = []
for itemList in soup.find_all('itemList') :
    gpsY = itemList.find('gpsY').string
    gpsX = itemList.find('gpsX').string

    busPos.append((gpsY, gpsX))

print('[ ' + busNum + '번 버스의 운행 위치 ]')
if len(busPos) != 0 :
    print(busNum + '번 버스 ' + str(len(busPos)) + '대 운행중...')
    for lat,lng in busPos :
        print(lat+','+lng)
else :
    print('현재 운행중인 ' + busNum + '번 버스가 없어요...')


In [ ]:
key = '796143536a756e69313134667752417a'
contentType = 'xml'
startIndex = '1'
endIndex = '100'
url = 'http://openapi.seoul.go.kr:8088/'+key+'/'+contentType+'/LampScpgmtb/'+startIndex+'/'+endIndex+'/'
savename = 'output/edu.xml'
req.urlretrieve(url, savename)

xml = open(savename, 'r', encoding='utf-8').read()
soup = BeautifulSoup(xml, 'xml')

pjList = []
for itemList in soup.find_all('row') :
    up_nm = itemList.find('UP_NM').string
    up_nm = '없음' if up_nm is None else up_nm
    pgm_nm = itemList.find('PGM_NM').string
    pgm_nm = '없음' if pgm_nm is None else pgm_nm
    target_nm = itemList.find('TARGET_NM').string
    target_nm = '없음' if target_nm is None else target_nm
    u_price = itemList.find('U_PRICE').string
    u_price = '없음' if u_price is None else u_price
    pjList.append((up_nm, pgm_nm, target_nm, u_price))

print('[ 서울 청소년 수련관 강좌 리스트 ]')
for up_nm,pgm_nm,target_nm,u_price in pjList : 
    print(up_nm+','+pgm_nm+','+target_nm+','+str(u_price))
    


In [ ]:
import random
print(random.randrange(1,11))
print(random.randrange(1,11))
print(random.randrange(1,11))
print(random.randrange(1,11))
print(random.randrange(1,11))
print(random.randrange(1,11))

In [ ]:
key = '74474a754173696e3533784e687a65'
contentType = 'xml'
url = 'http://openapi.seoul.go.kr:8088/74474a754173696e3533784e687a65/xml/SpotInfo/1/1000/'
res = req.urlopen(url)
soup = BeautifulSoup(res.read().decode('utf-8'), features="xml")
print(url)
#print(soup.prettify())
print(soup)

In [ ]:
spotinfo = []

for itemList in soup.find_all('row') :
    spot_nm = itemList.find('spot_nm').string   
    grs80tm_x = itemList.find('grs80tm_x').string    
    grs80tm_y = itemList.find('grs80tm_y').string    
    spotinfo.append((spot_nm, grs80tm_x, grs80tm_y))

In [ ]:
spotinfo = pd.DataFrame(spotinfo)
spotinfo.columns = ['spotname','tm_x','tm_y']
print(spotinfo)

In [ ]:
#tm좌표 위도,경도로 변환하기
proj_1 = Proj(init='epsg:2097')
proj_2 = Proj(init='epsg:4326')

DataFrame = spotinfo.copy()
x_list = []
y_list = []

for idx, row in DataFrame.iterrows():
    x, y = row['tm_x'], row['tm_y']
    x_, y_ = transform(proj_1, proj_2, x, y)
    x_list.append(x_)
    y_list.append(y_)
spotinfo['lon'] = x_list
spotinfo['lat'] = y_list

spotinfo.tail()
    

![NumPy image](images/서울시교통측정소.png)

In [ ]:
print(df)

In [ ]:
# spotinfo.to_csv('output/spotinfo.csv')

In [ ]:
mise = pd.read_excel('data/미세먼지 측정소별 교통량2015~.xlsx')
mise.drop('도로명 주소',axis=1,inplace=True)
print(mise)

In [ ]:
mise = mise.T


In [ ]:
test = mise[6]
test.drop('미세먼지 측정소 명',axis=0,inplace=True)
test = test.fillna(test.mean())
print(test)

In [ ]:
mist = mise
mist.drop('미세먼지 측정소 명',axis=0,inplace=True)
print(mist)

In [ ]:
mist = mist.fillna(mist.mean())

In [ ]:
print(mise['미세먼지 측정소 명'])
print(mist)

In [ ]:
test = mist.append(mise['미세먼지 측정소 명'])

In [ ]:
test = test.T
print(test)

In [ ]:
test = test[['미세먼지 측정소 명','2015양방','2016양방','2017양방','2018양방','2019양방','2020양방']]
test = pd.DataFrame(test)
print(test)
test.to_excel('output/미세먼지2015.xlsx')

In [187]:
rivers = pd.read_csv("data/rivers.csv",encoding='ANSI')
rivers.drop('순서',inplace=True,axis=1)
rivers.drop('시군구코드',inplace=True,axis=1)
rivers.drop('하천/호수일련번호',inplace=True,axis=1)
rivers.drop('하천/호수명_영문',inplace=True,axis=1)
rivers.replace(' ', np.nan, inplace=True)
rivers.dropna(inplace=True,axis=0)
river = rivers
print(river)

    하천/호수명_한글         위도          경도
9         고덕천  37.553192  127.172745
11        구기천  37.610200  126.956765
15         탄천  37.488995  127.094581
17        홍제천  37.600265  126.959566
29        중랑천  37.671566  127.052276
..        ...        ...         ...
540        탄천  37.479301  127.127848
578        탄천  37.482280  127.130417
591        탄천  37.472403  127.130754
592       고덕천  37.563663  127.175783
597       도림천  37.500737  126.891656

[112 rows x 3 columns]


In [191]:
print(river['하천/호수명_한글'].unique())

['고덕천' '구기천' '탄천' '홍제천' '중랑천' '우이천' '하천' '창릉천' '개봉천' '순원저수지' '정릉천' '태릉천'
 '한강' '여의천' '안양천' '도림천' '청계천' '안암천' '테스트' '불광천' '양재천' '일감호' '성내천' '새곡천'
 '인공폭포' '가양빗물펌프장' '반포천' '청룡상연못' '석촌호수']


In [197]:
import pandas as pd
import folium

# 미세먼지 측정소 리스트를 데이터프레임 변환
df = pd.read_excel('./data/미세먼지 측정소별 교통량.xlsx')
df = df[['미세먼지 측정소 명','도로명 주소','위도,경도']]
df['위도,경도'] = df['위도,경도'].replace(" ","")
latf = []
for i in df['위도,경도']:
    latf.append(i.split(','))
latlng = latf
lat = []
lng = []
for i in range(len(latlng)):
    lat.append(latlng[i][0])
    lng.append(latlng[i][1])
df['lat'] = lat
df['lng'] = lng

# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], 
                        zoom_start=12)

# 미세먼지 측정소를 CircleMarker로 표시
for name, lat, lng in zip(df['미세먼지 측정소 명'], df.lat,df.lng):
    folium.CircleMarker([lat,lng],
                        radius=4,         # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='red',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)

# 교통량 측정소를 표시    
for name, lat, lng in zip(spotinfo['spotname'], spotinfo['lat'], spotinfo['lon']):
    folium.CircleMarker([lat,lng],
                        radius=4,         # 원의 반지름
                        color='blue',         # 원의 둘레 색상
                        fill=True,
                        fill_color='blue',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)
    
for name, lat, lng in zip(river['하천/호수명_한글'], river['위도'], river['경도']):
    folium.CircleMarker([lat,lng],
                        radius=4,         # 원의 반지름
                        color='black',         # 원의 둘레 색상
                        fill=True,
                        fill_color='pink',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)

#빨간색은 미세먼지 측정소 위치, 파란색은 교통량 측정소 위치 
display(seoul_map)

# 지도를 HTML 파일로 저장하기
seoul_map.save('output/grennair_map.html')